In [ ]:
from pathlib import Path

import pandas as pd

from scraping_kit import load_db_and_bots

path_data = Path("data")
df_accounts = pd.read_excel(path_data / "twitter_accounts.xlsx")
df_accounts.sort_values("followersCount", ascending=False, inplace=True)
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

In [ ]:
list_screennames = df_accounts["screenName"].to_list()
MAX_WORKERS = 40
r = db_tw.collect_usertimeline(
    list_screennames,
    bots = bots,
    max_workers = MAX_WORKERS
)
